# Blockchain Implementation

### Sebastián Medina Carrillo & Cristian Camilo Lozano Jojoa
### Cybersecurity, Universidad Nacional de Colombia
A simple blockchain implementation with a client, miner and blockchain

### Creating the client that will generate transactions

In [89]:
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

## Creating the Client Class

In [2]:
class Client:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        self._signer  = PKCS1_v1_5.new(self._private_key)
        
    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
        

Creating an instance of a client and checking its identity

In [3]:
client = Client()
print(client.identity)

30819f300d06092a864886f70d010101050003818d0030818902818100c929443228e99d3d6b0f75e47f28ac4dec23d1942233eb420c1315eac4aff90e86d6fac3e64e28925420ba401d2cf66764ca98d5d2e912a83a6e9799177438452dc7f5ab01b9435d8922df7bc3caa09365a06ffda72c130d204618e3ca5f10ea8b844484891493ce0aae4284ae03f948eb3f33ec76217f1e10db729f20ae6c7f0203010001


### Creating a Transaction

In [4]:
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
        
    def to_dict(self):
        if self.sender == "Genesis":
            identity  = "Genesis"
        else:
            identity = self.sender.identity
        
        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time': self.time
        })
    
    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf-8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')
        

Testing the transaction Class

In [5]:
buyer = Client()
seller = Client()

transaction = Transaction(buyer,seller.identity,5.0)
signature = transaction.sign_transaction()
print(signature)

aec323c5b00fabc37b49f600eb6cac76ebf9146d56dcdd42248cc021f309e43f6c417f0dde7f6c760cb2de1f3713a83b8276025bac6cdb7d5a5c4520688270400ecde8514204f2c656896b248ab3fd2fa8cd4987ca6c030cfa30ad2cd66eb4478f34037dd3088d03ae908020ba15f569f9926aa7efaaabd025bde1af70557ac9


### Creating a helper function that displays the transaction contents

In [6]:
def display_transaction(transaction):
    dict = transaction.to_dict()
    print("Sender: "+ dict[('sender')])
    print("Recipient: "+ dict[('recipient')])
    print("Value: "+ str(dict[('value')]))
    print("Time: "+ str(dict[('time')]))
    print('----------------------')

Creating a list that holds transactions in a queue

In [7]:
transaction_queue = []

### Creating a few clients that will send and receive crypto

In [8]:
alan = Client()
dennis = Client()
ken = Client()
ada = Client()

### Creating a few transactions between clients

In [9]:
t1 = Transaction(alan,dennis.identity,15.0)
t1.sign_transaction()
transaction_queue.append(t1)
t2 = Transaction(dennis,ken.identity,10.0)
t2.sign_transaction()
transaction_queue.append(t2)
t3 = Transaction(ken,ada.identity,5.0)
t3.sign_transaction()
transaction_queue.append(t3)
t4 = Transaction(ada,alan.identity,3.0)
t4.sign_transaction()
transaction_queue.append(t4)


### Making a helper function that shows all transaction contents in the queue

In [10]:
def display_all(transaction_q):
    for transaction in transaction_q:
        display_transaction(transaction)
        print('----------------------')

In [11]:
display_all(transaction_queue)

Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100f531da91cd68da6cb5338cc28df104cd5ec7cb546ce8d29e02258e729c412174de866d4063e0ff9a5e0b37b844e1d1072b2dac7db8f2bcdd8e5905d828b80bc1f4e624f53f40b56d847309cb88788f19819d53bf4cbc944943388fec0797eddf01c4dd72c83b02e7b9325132d3befed7ef0871135b9bf02cb79e90736264f3e90203010001
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b31484ceceee80cf1d7b0b5f6af61e6721f065a9d5a6ca3f437e29c41ac6b7c846547b329a6ba1680e4dc4c4dc1d2c5bb9b5864a0607a3c5bba1374d48f844e96b5f60fdab86cdfa76ef39f4254b4a2a900a97e4ede4799a38c2633c61b27ed1c3c6ae741ca72498f32cac5c3e7e3d64bd3c6bc19099055af6fda43440120e6f0203010001
Value: 15.0
Time: 2019-07-17 20:19:43.554336
----------------------
----------------------
Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100b31484ceceee80cf1d7b0b5f6af61e6721f065a9d5a6ca3f437e29c41ac6b7c846547b329a6ba1680e4dc4c4dc1d2c5bb9b5864a0607a3c5bba1374d48f844e96b5f60fdab86cdfa76ef39f4254b4a2a900a97e4ede479

## Defining the Block Class

In [12]:
class Block:
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = ""
    

Creating a global variable with the last hash so that all blocks know

In [13]:
last_block_hash = ""

### Generating the Genesis Block

In [18]:
god = Client()
t0 = Transaction("Genesis",god.identity,1000.0)
block0 = Block()
block0.previous_block_hash = None
Nonce = None
block0.verified_transactions.append(t0)

### Calculating the block's digest and storing it to "last_block_hash"

In [19]:
digest = hash(block0)
last_block_hash = digest

## Creating our Cryptocurrency: Cybercoin

In [20]:
Cybercoin = []

### Creating a helper function that shows the whole blockchain

In [23]:
def show_blockchain(self):
    print("Numer of blocks in the chain: " + str(len(self)))
    for i in range(len(Cybercoin)):
        block_temp = Cybercoin[i]
        print("Block # " + str(i))
        for transaction in block_temp.verified_transactions:
            display_transaction(transaction)
            print('---------------------------')
        print("==============================")

### Adding the Genesis block to the Blockchain

In [24]:
Cybercoin.append(block0)
show_blockchain(Cybercoin)

Numer of blocks in the chain: 1
Block # 0
Sender: Genesis
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c09c39752809943457c1cf4c6af15aae73f05efd416a52dc24442f410513a1c634a7688eb6cd3203dfd7a12ec82f97b021696fa181ff58cdc9680eb5f30b3582984097ef18fc181ef6d2009faad9557fea363951af2395109539070a9fbf538a591df8e96ed234a59ad360ae658d83cef14f40433bac8e41af0367c2e8b18db50203010001
Value: 1000.0
Time: 2019-07-17 20:28:40.018259
----------------------
---------------------------


## Creating Miners, the Mining Function and Proof-of-Work

### Creating a helper function that generates a digest on a message

In [25]:
def sha256(msg):
    return hashlib.sha256(msg.encode('ascii')).hexdigest()

### Creating the mining function

In [50]:
def mine(msg, difficulty):
    assert difficulty >= 1
    found = False
    prefix = '1' * difficulty
    i = 0
    while found == False:
        i += 1
        digest = sha256(str(hash(msg)) + str(i))
        if digest.startswith(prefix):
            found = True
            print("Found Nonce: " + digest + ", after " + str(i) + " iterations.")
            return digest

In [55]:
mine("This is a test", 5)

Found Nonce: 11111c5650e7df74d112fe8239e8b3327c70ad77e564ddba527106720aad7b02, after 239430 iterations.


'11111c5650e7df74d112fe8239e8b3327c70ad77e564ddba527106720aad7b02'

### Adding a new Block to the Blockchain

In [57]:
last_transaction_index = 0
block = Block()
for i in range(3):
    temp_transaction = transaction_queue[last_transaction_index]
    block.verified_transactions.append(temp_transaction)
    last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine(block,2)
digest = hash(block)
Cybercoin.append(block)
last_block_hash = digest

Found Nonce: 11cdefa6a17ce2d6bb3a66ba14d2a88972a2cb31c68cd2b71252fd87ab33fac1, after 514 iterations.


### Showing the whole Blockchain

In [58]:
show_blockchain(Cybercoin)

Numer of blocks in the chain: 2
Block # 0
Sender: Genesis
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c09c39752809943457c1cf4c6af15aae73f05efd416a52dc24442f410513a1c634a7688eb6cd3203dfd7a12ec82f97b021696fa181ff58cdc9680eb5f30b3582984097ef18fc181ef6d2009faad9557fea363951af2395109539070a9fbf538a591df8e96ed234a59ad360ae658d83cef14f40433bac8e41af0367c2e8b18db50203010001
Value: 1000.0
Time: 2019-07-17 20:28:40.018259
----------------------
---------------------------
Block # 1
Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100f531da91cd68da6cb5338cc28df104cd5ec7cb546ce8d29e02258e729c412174de866d4063e0ff9a5e0b37b844e1d1072b2dac7db8f2bcdd8e5905d828b80bc1f4e624f53f40b56d847309cb88788f19819d53bf4cbc944943388fec0797eddf01c4dd72c83b02e7b9325132d3befed7ef0871135b9bf02cb79e90736264f3e90203010001
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b31484ceceee80cf1d7b0b5f6af61e6721f065a9d5a6ca3f437e29c41ac6b7c846547b329a6ba1680e4dc4c4dc1d2c5b

### Creating more transactions, and adding them to the queue

In [60]:
t5 = Transaction(ken,dennis.identity,25.0)
t5.sign_transaction()
transaction_queue.append(t5)
t6 = Transaction(ken,ada.identity,9.0)
t6.sign_transaction()
transaction_queue.append(t6)
t7 = Transaction(ada,alan.identity,13.0)
t7.sign_transaction()
transaction_queue.append(t7)

In [68]:
print(last_transaction_index)
display_all(transaction_queue)

7
Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100f531da91cd68da6cb5338cc28df104cd5ec7cb546ce8d29e02258e729c412174de866d4063e0ff9a5e0b37b844e1d1072b2dac7db8f2bcdd8e5905d828b80bc1f4e624f53f40b56d847309cb88788f19819d53bf4cbc944943388fec0797eddf01c4dd72c83b02e7b9325132d3befed7ef0871135b9bf02cb79e90736264f3e90203010001
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b31484ceceee80cf1d7b0b5f6af61e6721f065a9d5a6ca3f437e29c41ac6b7c846547b329a6ba1680e4dc4c4dc1d2c5bb9b5864a0607a3c5bba1374d48f844e96b5f60fdab86cdfa76ef39f4254b4a2a900a97e4ede4799a38c2633c61b27ed1c3c6ae741ca72498f32cac5c3e7e3d64bd3c6bc19099055af6fda43440120e6f0203010001
Value: 15.0
Time: 2019-07-17 20:19:43.554336
----------------------
----------------------
Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100b31484ceceee80cf1d7b0b5f6af61e6721f065a9d5a6ca3f437e29c41ac6b7c846547b329a6ba1680e4dc4c4dc1d2c5bb9b5864a0607a3c5bba1374d48f844e96b5f60fdab86cdfa76ef39f4254b4a2a900a97e4ede4

In [73]:
block = Block()
for i in range(4,7):
    temp = transaction_queue[last_transaction_index]
    block.verified_transactions.append(temp)
    last_transaction_index += 1
    
block.previous_block_hash = last_block_hash
block.Nonce = mine(block,2)
digest = hash(block)
Cybercoin.append(block)
last_block_hash = digest

Found Nonce: 11f1762c4dde7ee125094451a0ff600539536f54213b40bb4e25762e0601756e, after 515 iterations.


In [74]:
show_blockchain(Cybercoin)

Numer of blocks in the chain: 5
Block # 0
Sender: Genesis
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c09c39752809943457c1cf4c6af15aae73f05efd416a52dc24442f410513a1c634a7688eb6cd3203dfd7a12ec82f97b021696fa181ff58cdc9680eb5f30b3582984097ef18fc181ef6d2009faad9557fea363951af2395109539070a9fbf538a591df8e96ed234a59ad360ae658d83cef14f40433bac8e41af0367c2e8b18db50203010001
Value: 1000.0
Time: 2019-07-17 20:28:40.018259
----------------------
---------------------------
Block # 1
Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100f531da91cd68da6cb5338cc28df104cd5ec7cb546ce8d29e02258e729c412174de866d4063e0ff9a5e0b37b844e1d1072b2dac7db8f2bcdd8e5905d828b80bc1f4e624f53f40b56d847309cb88788f19819d53bf4cbc944943388fec0797eddf01c4dd72c83b02e7b9325132d3befed7ef0871135b9bf02cb79e90736264f3e90203010001
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b31484ceceee80cf1d7b0b5f6af61e6721f065a9d5a6ca3f437e29c41ac6b7c846547b329a6ba1680e4dc4c4dc1d2c5b

### Creating a function so we can add a block easier

In [85]:
def add_block(block,size,index,last_block_hash):
    last_transaction_index = index
    for i in range(size):
        temp = transaction_queue[last_transaction_index]
        block.verified_transactions.append(temp)
        last_transaction_index += 1
    block.previous_block_hash = last_block_hash
    block.Nonce = mine(block,5)
    digest = hash(block)
    Cybercoin.append(block)
    last_block_hash = digest
    show_blockchain(Cybercoin)    
        
        

In [84]:
t8 = Transaction(alan,ada.identity,1.0)
t8.sign_transaction()
transaction_queue.append(t8)
block = Block()
#display_all(transaction_queue)
add_block(block,1,8,last_block_hash)

Found Nonce: 11b9a7864abd1e736ff1b48fb0abd34aa16e365e4d13b3b28a620c3065135474, after 1282 iterations.
Numer of blocks in the chain: 8
Block # 0
Sender: Genesis
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c09c39752809943457c1cf4c6af15aae73f05efd416a52dc24442f410513a1c634a7688eb6cd3203dfd7a12ec82f97b021696fa181ff58cdc9680eb5f30b3582984097ef18fc181ef6d2009faad9557fea363951af2395109539070a9fbf538a591df8e96ed234a59ad360ae658d83cef14f40433bac8e41af0367c2e8b18db50203010001
Value: 1000.0
Time: 2019-07-17 20:28:40.018259
----------------------
---------------------------
Block # 1
Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100f531da91cd68da6cb5338cc28df104cd5ec7cb546ce8d29e02258e729c412174de866d4063e0ff9a5e0b37b844e1d1072b2dac7db8f2bcdd8e5905d828b80bc1f4e624f53f40b56d847309cb88788f19819d53bf4cbc944943388fec0797eddf01c4dd72c83b02e7b9325132d3befed7ef0871135b9bf02cb79e90736264f3e90203010001
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902

### Transaction lifecycle example

In [88]:
cristian = Client()
sebastian = Client()
t9 = Transaction(cristian,sebastian.identity,4)
t9.sign_transaction()
transaction_queue.append(t9)
new_block = Block()
add_block(new_block,1,-1,last_block_hash)

Found Nonce: 11111bf3e0989a0d7e47b1e1a6212df39b2fcdd766a0210ea32a5da5de69ce32, after 3128298 iterations.
Numer of blocks in the chain: 9
Block # 0
Sender: Genesis
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100c09c39752809943457c1cf4c6af15aae73f05efd416a52dc24442f410513a1c634a7688eb6cd3203dfd7a12ec82f97b021696fa181ff58cdc9680eb5f30b3582984097ef18fc181ef6d2009faad9557fea363951af2395109539070a9fbf538a591df8e96ed234a59ad360ae658d83cef14f40433bac8e41af0367c2e8b18db50203010001
Value: 1000.0
Time: 2019-07-17 20:28:40.018259
----------------------
---------------------------
Block # 1
Sender: 30819f300d06092a864886f70d010101050003818d0030818902818100f531da91cd68da6cb5338cc28df104cd5ec7cb546ce8d29e02258e729c412174de866d4063e0ff9a5e0b37b844e1d1072b2dac7db8f2bcdd8e5905d828b80bc1f4e624f53f40b56d847309cb88788f19819d53bf4cbc944943388fec0797eddf01c4dd72c83b02e7b9325132d3befed7ef0871135b9bf02cb79e90736264f3e90203010001
Recipient: 30819f300d06092a864886f70d010101050003818d0030818